In [ ]:
from langchain_community.document_loaders       import TextLoader
from langchain_community.vectorstores           import FAISS
from langchain_community.embeddings             import OllamaEmbeddings
from langchain.text_splitter                    import CharacterTextSplitter

## Text Splitting

In [ ]:
loader      = TextLoader('data/speech.txt')
documents   = loader.load()

In [ ]:
textSplitter    = CharacterTextSplitter(
    chunk_size      = 1000,
    chunk_overlap   = 30
)

document = textSplitter.split_documents(documents)

In [ ]:
document

## Embeddings

In [ ]:
embeddings      = OllamaEmbeddings(model="gemma:2b")

database        = FAISS.from_documents(
    documents,
    embeddings
)

In [ ]:
database

In [ ]:
query = "How does the speaker describe the desired outcome of the war?"

result = database.similarity_search(query)

In [ ]:
print(f'Result:         {result}')
print(f'First Result:   {result[0].page_content}')

## Retriever

In [ ]:
retriever = database.as_retriever()

result = retriever.invoke(query)
print(f'First Result:   {result[0].page_content}')

## Similarity Search with Score

In [ ]:
similarityScoreForDocument = database.similarity_search_with_score(query)
similarityScoreForDocument

## Passing Vectors instead of Sentences

In [ ]:
embeddingVector = embeddings.embed_query(query)
embeddingVector

In [ ]:
result = database.similarity_search_with_score(embeddingVector)
result

## Save and Load the Database

In [ ]:
database.save_local('faiss_index')

In [ ]:
newDatabase = FAISS.load_local(
    'faiss_index', 
    embeddings,
    allow_dangerous_deserialization = True
)

In [ ]:
documents = newDatabase.similarity_search(query)
documents